Please **submit this Jupyter notebook through Canvas** no later than **Monday November 19, 12:59**, before the start of the lecture.

Homework is in **groups of two**, and you are expected to hand in original work. Work that is copied from another group will not be accepted.

# Exercise 0
Write down the names + student ID of the people in your group.

**Jordan Earle** (12297127)

**Natasja Wezel** (11027649)

In [2]:
from scipy.sparse import diags
import scipy.linalg
import scipy as sp
import matplotlib.pyplot as plt
import numpy as np

-------------
# Exercise 1
# (a)
Let $Q$ be a real, square orthogonal matrix. Show that all eigenvalues of $Q$ satisfy $|\lambda |=1$.

If $\lambda$ is an eigenvalue of Q with corresponding eigenvector $x$, $Qx = \lambda x$. This would mean that $||Qx||_2 = ||\lambda x||_2$.  An orthoginal matrix such as Q, operating an vector x, would only cause a rotation, reflection or retroreflection, and since an orthogonal matrix preserves the 2-norm, we have $||x||_2 = \lambda \cdot ||x||_2$, which would mean that $|\lambda| = 1$.

$$
\begin{align}
    Qx &= \lambda x\\
    ||Qx||_2 &= ||\lambda x||_2\\
    ||x||_2 &= \lambda \cdot ||x||_2
\end{align}
$$


# (b)
A projection matrix is a matrix $P$ for which $P^2=P$. Find the eigenvalues of a projection matrix.

If $\lambda$ is an eigenvalue of P with corresponding eigenvector $x$, $Px = \lambda x$. This would mean that $P^2x = \lambda Px = \lambda^2 x$. This means that $\lambda^2$ is an eigenvalue of $P^2$. If $P = P^2$, the eigenvalues are thus such that $\lambda^2 = \lambda$. The only possible values which satisfy this are 0 and 1.


$$
\begin{align}
    Px &= \lambda x\\
    PPx &= P \lambda x\\
    P^2x &= \lambda Px\\
    P^2 x &= \lambda^2 x
\end{align}
$$

-----------
# Exercise 2
Generate a symmetric $4 × 4$ matrix $A$ randomly, e.g. as below.
Implement Rayleigh quotient iteration (Algorithm 4.4) to compute the largest eigenvalue and corresponding eigenvector of $A$. Use a suitable stopping criterion for your iterations (and explain why you think it is suitable). You may use `scipy.linalg.solve`.

If $x$ is an approximate eigenvector for a real matrix $A$, then determining the best estimate for the corresponding eigenvalue $\lambda$ can be considered as an $n \times 1$ linear least squares approximate problem: $x \lambda \cong Ax$. The matrix has a normal equation $x^Tx\lambda = x^TAx$ and from this we can see that the least squares solution is given by:

$$
\lambda = \frac{x^TAx}{x^Tx}
$$

This quantity is known as the *Rayleigh quotient* and can be used to accelerate the convergence of a method such as power iteration, since at iteration *k* 

In [7]:
B = np.random.randn(4,4)
A = B+B.T
#A = np.array([[3.,1.], [1., 3.]])
I = np.eye(A.shape[0])
# Need to add in a check to make sure the matrix is ok, sometimes it crashes
#print(I)
#print(A)

x = np.random.rand(A.shape[0],1)

escbreak = True

prev_sigma = 0

for potatto in range(0, 10):
    x = A.dot(x)

while escbreak:
    
    xT = x.T
    # compute shift
    sigma = xT.dot(A.dot(x))/(xT.dot(x))
    left = (A - sigma*I)
    
    # generate next vector
    ycur = np.linalg.solve(left, x)
    
    # normalize the vector
    x = ycur/np.linalg.norm(ycur, ord = 2)

    # check to see if the loop should break 
    
    if (np.linalg.norm(A.dot(x), ord = 2) - np.linalg.norm(sigma*x, ord = 2)) < 0.0001:
        escbreak = False
    prev_sigma = sigma

# find actual eigenvalues    
w, v = sp.linalg.eig(A)
wabs = abs(w.real)
max_eig = max(wabs)
loc = np.where(wabs == max_eig)
max_vex = v[loc]
max_val = w[loc].real

# print the matrix, eigenvalue and eigenvector
print("For the matrix: \n", A)
print("The dominant Eigenvalue is approx: ", sigma)
print("The dominant Eigenvector is approx: \n", x)
print('The true dominant eigenvalue is \n', max_val)
print("The true dominant eigenvector is \n", max_vex)

For the matrix: 
 [[-3.44610212  3.39605219  2.43548198  2.09382226]
 [ 3.39605219  1.08165391  0.89106994 -0.86282963]
 [ 2.43548198  0.89106994  1.51756459  0.07548154]
 [ 2.09382226 -0.86282963  0.07548154  2.18030895]]
The dominant Eigenvalue is approx:  [[-6.26604945]]
The dominant Eigenvector is approx: 
 [[-0.85481402]
 [ 0.39789707]
 [ 0.21948862]
 [ 0.25059062]]
The true dominant eigenvalue is 
 [-6.26605121]
The true dominant eigenvector is 
 [[-0.85481402  0.49735283  0.12873873  0.07320857]]


In order to determine a stopping point, a number of methods were discussed. The first and most obvious is that if the value of sigma is not changing much (i.e. less that 0.0001).  This would indicated convergence and was the method used.  Another stopping point would be to stop the system when the determinant of $A-\sigma I$ becomes 0 as that would indicate that it is singular and that we have found the eigenvalue.  This could be dangerous though if the system does not converge to the exact eigenvalue quickly.  One more method discussed was checking to see if $||Ax||_2 - ||\lambda x||_2 < certainty value$.  This was implemented as it was more acccurate than trying checking if the value of lambada was not changing quickly.  

-------------
# Exercise 3
Let $A$ be a $n × n$ tridiagonal matrix that results (up to an overall constant scaling factor) from an
$n$-point spatial discretization of the Laplace operator in $1$ spatial dimension. Specifically, all diagonal
elements of $A$ have the value $-2$, all elements just below or above the diagonal have value $1$.

$${\displaystyle A = \left( \begin{array}{rrrrr} -2 &  1  &  0 &  0& \ldots \\ 1  & -2 & 1 & 0 &\ldots   \\ 0 & 1 & -2 &  1 &\ldots \\ \vdots & & & & \ddots \end{array} \right) \,}$$



Take $n = 150$. Use `scipy.linalg.qr` to implement Algorithm 4.7 from the book and compute the full eigenvalue spectrum of $A$. Compare against
the eigenvalues obtained by the built-in routine `scipy.linalg.eig(A)`. Show in a figure the (approximate)
eigenvalues after 10, 100 and 500 QR iterations, as well as the eigenvalues obtained with `eig(A)`.


In [ ]:
n = 150
A = diags([1, -2, 1], [-1, 0, 1], shape=(n, n)).toarray()
w, v = sp.linalg.eig(A)

plt.figure(0)
plt.title("True Eigenvalues")
plt.hist(w, bins=50)

iters = [10,100,150,300,500, 1000, 2000]

# cycle through the iterations
for i in iters:
    # reset the matrix A at the start of each iteration
    plt.figure(i)
    
    for k in range(0,i):
        Q, R = sp.linalg.qr(A)
        A = R.dot(Q)
    
    # find the eigenvalues
    calceig = []
    
    for j in range(0, A.shape[0]):
        calceig.append(A[j,j])
    
    #plt.hist(calceig, bins=50)
    #plt.title("Eigenvalues with " + str(i) + " iterations")
    
    f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(15,5))
    ax1.set_title("True eigenvalues")
    ax2.set_title("Eigenvalues with " + str(i) + " iterations")
    ax1.hist(w, bins=50)
    ax2.hist(calceig,bins=50)

# we can ignore the warning about casting discards the imaginary part because the imaginary part is 0 anyway

In [ ]:
n = 150
A0 = diags([1, -2, 1], [-1, 0, 1], shape=(n, n)).toarray()
w, v = sp.linalg.eig(A0)

w = np.sort(w.real)

plt.figure(figsize=(15,15))
plt.plot(range(0,150), w, label="True eigenvalues")
plt.title("First 150 Eigenvalues of the Matrix")
plt.xlabel('Eigenvalue Number')
plt.ylabel('Value of Eignenvalue')

iters = [10,100,150,300,500, 1000, 2000]
eigniters = []
# cycle through the iterations
for i in iters:
    # reset the matrix A at the start of each iteration
    A = A0
    
    for k in range(0,i):
        Q, R = sp.linalg.qr(A)
        A = R.dot(Q)
    
    # find the eigenvalues
    calceig = []
    
    for j in range(0, A.shape[0]):
        calceig.append(A[j,j])
    eigniters.append(calceig)
    plt.plot(range(0,150), calceig, label="Eigenvalues with " + str(i) + " iterations", alpha=.40)

plt.legend()

In [ ]:
plt.figure(figsize=(15,15))
plt.title("Absolute error in calculated eigenvalues")
for i in range(len(iters)):
    value = abs(w- eigniters[i])
    plt.plot(value, label="Eigenvalues with " + str(iters[i]) + " iterations")
plt.legend()